<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-pyspark-и-библиотек.-Создание-spark-сессии" data-toc-modified-id="Импорт-pyspark-и-библиотек.-Создание-spark-сессии-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт pyspark и библиотек. Создание spark сессии</a></span></li><li><span><a href="#Витрина-corporate_payments" data-toc-modified-id="Витрина-corporate_payments-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Витрина corporate_payments</a></span><ul class="toc-item"><li><span><a href="#Считывание-витрины-corporate_payments-в-датафрейм" data-toc-modified-id="Считывание-витрины-corporate_payments-в-датафрейм-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Считывание витрины corporate_payments в датафрейм</a></span></li><li><span><a href="#Запись-витрины-corporate_payments-в-parquet-файлы-с-партицированием-по-дате-операции" data-toc-modified-id="Запись-витрины-corporate_payments-в-parquet-файлы-с-партицированием-по-дате-операции-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Запись витрины corporate_payments в parquet файлы с партицированием по дате операции</a></span></li><li><span><a href="#Вывод-витрины-corporate_payments-на-2020-11-01-из-файла-parquet" data-toc-modified-id="Вывод-витрины-corporate_payments-на-2020-11-01-из-файла-parquet-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Вывод витрины corporate_payments на 2020-11-01 из файла parquet</a></span></li></ul></li><li><span><a href="#Витрина-corporate_account" data-toc-modified-id="Витрина-corporate_account-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Витрина corporate_account</a></span></li><li><span><a href="#Витрина-corporate_info" data-toc-modified-id="Витрина-corporate_info-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Витрина corporate_info</a></span></li></ul></div>

# Импорт pyspark и библиотек. Создание spark сессии

In [1]:
import findspark
findspark.init('/opt/spark-3.3.1')
import pyspark
sc = pyspark.SparkContext()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F 

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "/opt/spark-3.3.1/jars/postgresql-42.5.1.jar") \
    .config("spark.driver.bindAddress","localhost") \
    .config("spark.ui.port","5432") \
    .config("parquet.encoding","UTF-8") \
    .getOrCreate()

# Витрина corporate_payments

## Считывание витрины corporate_payments в датафрейм 

In [4]:
df_corporate_payments = spark.read.format("jdbc")\
.option("url", "jdbc:postgresql://localhost:5432/de_project4") \
.option("driver", "org.postgresql.Driver")\
.option("user", "postgres").option("password", "1")\
.option("query", "select * from corporate_payments")\
.load()

In [5]:
df_corporate_payments.printSchema()

root
 |-- AccountId: integer (nullable = true)
 |-- ClientId: integer (nullable = true)
 |-- PaymentAmt: decimal(38,18) (nullable = true)
 |-- EnrollementAmt: decimal(38,18) (nullable = true)
 |-- TaxAmt: decimal(38,18) (nullable = true)
 |-- ClearAmt: decimal(38,18) (nullable = true)
 |-- CarsAmt: decimal(38,18) (nullable = true)
 |-- FoodAmt: decimal(38,18) (nullable = true)
 |-- CutoffDt: date (nullable = true)



In [6]:
df_corporate_payments=df_corporate_payments.withColumn("PaymentAmt",F.round("PaymentAmt",2))
df_corporate_payments=df_corporate_payments.withColumn("EnrollementAmt",F.round("EnrollementAmt",2))
df_corporate_payments=df_corporate_payments.withColumn("TaxAmt",F.round("TaxAmt",2))
df_corporate_payments=df_corporate_payments.withColumn("ClearAmt",F.round("ClearAmt",2))
df_corporate_payments=df_corporate_payments.withColumn("CarsAmt",F.round("CarsAmt",2))
df_corporate_payments=df_corporate_payments.withColumn("FoodAmt",F.round("FoodAmt",2))

In [7]:
df_corporate_payments.show()

+---------+--------+----------+--------------+----------+----------+----------+---------+----------+
|AccountId|ClientId|PaymentAmt|EnrollementAmt|    TaxAmt|  ClearAmt|   CarsAmt|  FoodAmt|  CutoffDt|
+---------+--------+----------+--------------+----------+----------+----------+---------+----------+
|        1|       1|1239646.48|    1697988.80|      null|      null|1239646.48|722086.05|2020-11-02|
|        2|       2| 856510.75|    1504469.43|      null|      null| 856510.75|     null|2020-11-03|
|        3|       3|1354023.20|    1110291.49|1354023.20|      null|1354023.20|561642.09|2020-11-04|
|        4|       4|1239725.91|    1698092.00|1239725.91|      null|1239725.91|     null|2020-11-01|
|        5|       5| 856586.87|    1504548.86|      null|      null| 856586.87|401198.14|2020-11-02|
|        6|       6|1354126.40|    1110367.61|1354126.40|1110367.61|1354126.40|     null|2020-11-03|
|        7|       7|1239805.34|    1698195.20|1239805.34|      null|1239805.34|240754.18|20

In [8]:
df_corporate_payments.printSchema()

root
 |-- AccountId: integer (nullable = true)
 |-- ClientId: integer (nullable = true)
 |-- PaymentAmt: decimal(23,2) (nullable = true)
 |-- EnrollementAmt: decimal(23,2) (nullable = true)
 |-- TaxAmt: decimal(23,2) (nullable = true)
 |-- ClearAmt: decimal(23,2) (nullable = true)
 |-- CarsAmt: decimal(23,2) (nullable = true)
 |-- FoodAmt: decimal(23,2) (nullable = true)
 |-- CutoffDt: date (nullable = true)



## Запись витрины corporate_payments в parquet файлы с партицированием по дате операции

In [9]:
df_corporate_payments.write.option("encoding","UTF-8").mode("overwrite").partitionBy("CutoffDt").parquet("corporate_payments")

## Вывод витрины corporate_payments на 2020-11-01 из файла parquet

In [22]:
df_corporate_payments_20201101=spark.read.parquet("corporate_payments/CutoffDt=2020-11-01")

In [23]:
df_corporate_payments_20201101.show()

+---------+--------+----------+--------------+----------+----------+----------+----------+
|AccountId|ClientId|PaymentAmt|EnrollementAmt|    TaxAmt|  ClearAmt|   CarsAmt|   FoodAmt|
+---------+--------+----------+--------------+----------+----------+----------+----------+
|     5003|    5003| 983401.12|    2493467.77| 983401.12| 856586.87| 983401.12| 481420.12|
|     5004|    5004|1526057.60|    2591308.27|1526057.60|1354126.40|1526057.60| 120385.12|
|     5007|    5007|1526160.80|    2591487.58|1526160.80|1354229.60|1526160.80| 762321.39|
|     5008|    5008|1372216.82|    3110114.37|1372216.82|3110114.37|1372216.82|      null|
|     5011|    5011|1372296.25|    3110297.00|      null|      null|1372296.25| 280901.28|
|     5012|    5012| 983629.48|    2493934.42|      null|      null| 983629.48| 481492.32|
|     5015|    5015| 983705.60|    2494089.97| 983705.60|      null| 983705.60|      null|
|     5016|    5016|1526470.40|    2592025.54|1526470.40|      null|1526470.40| 762465.81|

# Витрина corporate_account

In [12]:
df_corporate_account = spark.read.format("jdbc")\
.option("url", "jdbc:postgresql://localhost:5432/de_project4") \
.option("driver", "org.postgresql.Driver")\
.option("user", "postgres").option("password", "1")\
.option("query", "select * from corporate_account")\
.load()

In [13]:
df_corporate_account.printSchema()

root
 |-- AccountId: integer (nullable = true)
 |-- AccountNum: string (nullable = true)
 |-- DateOpen: date (nullable = true)
 |-- ClientId: integer (nullable = true)
 |-- ClientName: string (nullable = true)
 |-- TotalAmt: decimal(38,18) (nullable = true)
 |-- CutoffDt: date (nullable = true)



In [14]:
df_corporate_account=df_corporate_account.withColumn("TotalAmt",F.round("TotalAmt",2))
df_corporate_account.printSchema()

root
 |-- AccountId: integer (nullable = true)
 |-- AccountNum: string (nullable = true)
 |-- DateOpen: date (nullable = true)
 |-- ClientId: integer (nullable = true)
 |-- ClientName: string (nullable = true)
 |-- TotalAmt: decimal(23,2) (nullable = true)
 |-- CutoffDt: date (nullable = true)



In [15]:
df_corporate_account.write.option("encoding","UTF-8").mode("overwrite").partitionBy("CutoffDt").parquet("corporate_account")

In [16]:
df_corporate_account_20201101=spark.read.parquet("corporate_account/CutoffDt=2020-11-01")
df_corporate_account_20201101.show()

+---------+--------------------+----------+--------+-----------+----------+
|AccountId|          AccountNum|  DateOpen|ClientId| ClientName|  TotalAmt|
+---------+--------------------+----------+--------+-----------+----------+
|     5003|40802810520270000061|2020-11-03|    5003|Клиент 5003|3476868.89|
|     5004|40702810623050000084|2020-11-04|    5004|Клиент 5004|4117365.87|
|     5007|40702810320640000569|2020-11-03|    5007|Клиент 5007|4117648.38|
|     5008|40802810610450000337|2020-11-04|    5008|Клиент 5008|4482331.19|
|     5011|40702810414030000268|2020-11-03|    5011|Клиент 5011|4482593.25|
|     5012|40802810509560000364|2020-11-04|    5012|Клиент 5012|3477563.90|
|     5015|40802810107250000129|2020-11-03|    5015|Клиент 5015|3477795.57|
|     5016|40802810220110000260|2020-11-04|    5016|Клиент 5016|4118495.94|
|     5019|40802810222180000263|2020-11-03|    5019|Клиент 5019|4118778.46|
|     5020|40702810144410000147|2020-11-04|    5020|Клиент 5020|4483379.43|
|     5023|4

# Витрина corporate_info

In [17]:
df_corporate_info = spark.read.format("jdbc")\
.option("url", "jdbc:postgresql://localhost:5432/de_project4") \
.option("driver", "org.postgresql.Driver")\
.option("user", "postgres").option("password", "1")\
.option("query", "select * from corporate_info")\
.load()

In [18]:
df_corporate_info.printSchema()

root
 |-- clientid: integer (nullable = true)
 |-- clientname: string (nullable = true)
 |-- type: string (nullable = true)
 |-- form: string (nullable = true)
 |-- registerdate: date (nullable = true)
 |-- TotalAmt: decimal(38,18) (nullable = true)
 |-- CutoffDt: date (nullable = true)



In [19]:
df_corporate_info=df_corporate_info.withColumn("TotalAmt",F.round("TotalAmt",2))
df_corporate_info.printSchema()

root
 |-- clientid: integer (nullable = true)
 |-- clientname: string (nullable = true)
 |-- type: string (nullable = true)
 |-- form: string (nullable = true)
 |-- registerdate: date (nullable = true)
 |-- TotalAmt: decimal(23,2) (nullable = true)
 |-- CutoffDt: date (nullable = true)



In [20]:
df_corporate_info.write.option("encoding","UTF-8").mode("overwrite").partitionBy("CutoffDt").parquet("corporate_info")

In [21]:
df_corporate_info_20201101=spark.read.parquet("corporate_account/CutoffDt=2020-11-01")
df_corporate_info_20201101.show()

+---------+--------------------+----------+--------+-----------+----------+
|AccountId|          AccountNum|  DateOpen|ClientId| ClientName|  TotalAmt|
+---------+--------------------+----------+--------+-----------+----------+
|     5003|40802810520270000061|2020-11-03|    5003|Клиент 5003|3476868.89|
|     5004|40702810623050000084|2020-11-04|    5004|Клиент 5004|4117365.87|
|     5007|40702810320640000569|2020-11-03|    5007|Клиент 5007|4117648.38|
|     5008|40802810610450000337|2020-11-04|    5008|Клиент 5008|4482331.19|
|     5011|40702810414030000268|2020-11-03|    5011|Клиент 5011|4482593.25|
|     5012|40802810509560000364|2020-11-04|    5012|Клиент 5012|3477563.90|
|     5015|40802810107250000129|2020-11-03|    5015|Клиент 5015|3477795.57|
|     5016|40802810220110000260|2020-11-04|    5016|Клиент 5016|4118495.94|
|     5019|40802810222180000263|2020-11-03|    5019|Клиент 5019|4118778.46|
|     5020|40702810144410000147|2020-11-04|    5020|Клиент 5020|4483379.43|
|     5023|4